# Deploying Iris-detection model using Vertex AI


## Overview

In this tutorial, you build a scikit-learn model and deploy it on Vertex AI using the custom container method. You use the FastAPI Python web server framework to create a prediction endpoint. You also incorporate a preprocessor from training pipeline into your online serving application.

Learn more about [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training) and [Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions).

### Objective

In this notebook, you learn how to create, deploy and serve a custom classification model on Vertex AI. This notebook focuses more on deploying the model than on the design of the model itself. 


This tutorial uses the following Vertex AI services and resources:

- Vertex AI models
- Vertex AI endpoints

The steps performed include:

- Train a model that uses flower's measurements as input to predict the class of iris.
- Save the model and its serialized pre-processor.
- Build a FastAPI server to handle predictions and health checks.
- Build a custom container with model artifacts.
- Upload and deploy custom container to Vertex AI Endpoints.

### Dataset

This tutorial uses R.A. Fisher's Iris dataset, a small and popular dataset for machine learning experiments. Each instance has four numerical features, which are different measurements of a flower, and a target label that
categorizes the flower into: **Iris setosa**, **Iris versicolour** and **Iris virginica**.

This tutorial uses [a version of the Iris dataset available in the
scikit-learn library](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris).

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Artifact Registry
* Cloud Build

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Artifact Registry pricing](https://cloud.google.com/artifact-registry/pricing) and [Cloud Build pricing](https://cloud.google.com/build/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages



In [1]:

# Vertex SDK for Python
! pip3 install --upgrade --quiet  google-cloud-aiplatform

### Set Google Cloud project information 
Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "wise-ally-461615-t2"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [3]:
BUCKET_URI = f"gs://week1_assignment_1_lab_demo_iitm_bs"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [4]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://week1_assignment_1_lab_demo_iitm_bs/...
ServiceException: 409 A Cloud Storage bucket named 'week1_assignment_1_lab_demo_iitm_bs' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). 

In [5]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import the required libraries

In [6]:
import os
import sys

### Configure resource names

Set a name for the following parameters:

`MODEL_ARTIFACT_DIR` - Folder directory path to your model artifacts within a Cloud Storage bucket, for example: "my-models/fraud-detection/trial-4"

`REPOSITORY` - Name of the Artifact Repository to create or use.

`IMAGE` - Name of the container image that is pushed to the repository.

`MODEL_DISPLAY_NAME` - Display name of Vertex AI model resource.

In [7]:
MODEL_ARTIFACT_DIR = "my-models/iris-classifier-week-1"  # @param {type:"string"}
REPOSITORY = "iris-classifier-repo"  # @param {type:"string"}
IMAGE = "iris-classifier-img"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "iris-classifier"  # @param {type:"string"}

# Set the defaults if no names were specified
if MODEL_ARTIFACT_DIR == "[your-artifact-directory]":
    MODEL_ARTIFACT_DIR = "custom-container-prediction-model"

if REPOSITORY == "[your-repository-name]":
    REPOSITORY = "custom-container-prediction"

if IMAGE == "[your-image-name]":
    IMAGE = "sklearn-fastapi-server"

if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "sklearn-custom-container"

## Simple Decision Tree model
Build a Decision Tree model on iris data

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas.plotting import parallel_coordinates
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics


data = pd.read_csv('dataset/iris.csv')
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [9]:
train, test = train_test_split(data, test_size = 0.4, stratify = data['species'], random_state = 42)
X_train = train[['sepal_length','sepal_width','petal_length','petal_width']]
y_train = train.species
X_test = test[['sepal_length','sepal_width','petal_length','petal_width']]
y_test = test.species

In [10]:
mod_dt = DecisionTreeClassifier(max_depth = 3, random_state = 1)
mod_dt.fit(X_train,y_train)
prediction=mod_dt.predict(X_test)
print('The accuracy of the Decision Tree is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))

The accuracy of the Decision Tree is 0.983


In [11]:
import pickle
import joblib

joblib.dump(mod_dt, "artifacts/model.joblib")

['artifacts/model.joblib']

### Upload model artifacts and custom code to Cloud Storage

Before you can deploy your model for serving, Vertex AI needs access to the following files in Cloud Storage:

* `model.joblib` (model artifact)
* `preprocessor.pkl` (model artifact)

Run the following commands to upload your files:

In [12]:
!gsutil cp artifacts/model.joblib {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://artifacts/model.joblib [Content-Type=application/octet-stream]...
/ [1 files][  2.5 KiB/  2.5 KiB]                                                
Operation completed over 1 objects/2.5 KiB.                                      


In [13]:
!git config --global user.name "SakethPatel"

In [14]:
!dvc add dataset/iris.csv


⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/jupyter/.dvc/cache/files/md5'| |0/? [00:00<?,    ?
                                                                                
!
  0%|          |Checking out /home/jupyter/dataset/iri0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 30.21file/s]

To track the changes with git, run:

	git add dataset/iris.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [15]:
!git add dataset/iris.csv.dvc .gitignore
!git commit -m "Track iris dataset with DVC"


fatal: pathspec '.gitignore' did not match any files
On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.bashrc
	.cache/
	.config/
	.docker/
	.gitconfig
	.gsutil/
	.ipynb_checkpoints/
	.ipython/
	.jupyter/
	.local/
	.npm/
	21F2000054_Assignment_1_MAY_2025_MLOps.ipynb
	artifacts/
	dataset/
	notebook_template.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [16]:
BUCKET = "gs://week1_assignment_2_lab_demo_iitm_bs"
!gsutil mb -l us-central1 {BUCKET}


Creating gs://week1_assignment_2_lab_demo_iitm_bs/...
ServiceException: 409 A Cloud Storage bucket named 'week1_assignment_2_lab_demo_iitm_bs' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [17]:
!dvc remote add -d myremote {BUCKET}


Setting 'myremote' as a default remote.
ERROR: configuration error - config file error: remote 'myremote' already exists. Use `-f|--force` to overwrite it.


In [18]:
!dvc push


Pushing
!
  0% Checking cache in 'week1_assignment_2_lab_demo_iitm_bs/files/md5'| |0/? [00
                                                                                
!
  0% Checking cache in '/home/jupyter/.dvc/cache/files/md5'| |0/? [00:00<?,    ?
                                                                                
!
  0%|          |Pushing to gs                         0/1 [00:00<?,     ?file/s]
  0%|          |Pushing to gs                         0/1 [00:00<?,     ?file/s]

!

  0%|          |/home/jupyter/.dvc/cache/files/0.00/3.77k [00:00<?,        ?B/s]

                                                                                
Pushing                                                                         
1 file pushed


In [19]:
!git add .dvc/config
!git commit -m "Configure DVC remote as GCS"


On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.bashrc
	.cache/
	.config/
	.docker/
	.gitconfig
	.gsutil/
	.ipynb_checkpoints/
	.ipython/
	.jupyter/
	.local/
	.npm/
	21F2000054_Assignment_1_MAY_2025_MLOps.ipynb
	artifacts/
	dataset/
	notebook_template.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [24]:
!dvc stage add -n train_model \
  -d dataset/iris.csv -d train.py \
  -o artifacts/model.joblib \
  python train.py

!git add dvc.yaml dvc.lock
!git commit -m "Add training pipeline to DVC"


Added stage 'train_model' in 'dvc.yaml'                               core>

To track the changes with git, run:

	git add dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true
fatal: pathspec 'dvc.lock' did not match any files
On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.bashrc
	.cache/
	.config/
	.docker/
	.gitconfig
	.gsutil/
	.ipynb_checkpoints/
	.ipython/
	.jupyter/
	.local/
	.npm/
	21F2000054_Assignment_1_MAY_2025_MLOps.ipynb
	artifacts/
	dataset/
	dvc.yaml
	notebook_template.ipynb
	train.py

nothing added to commit but untracked files present (use "git add" to track)


In [25]:
!dvc repro


'dataset/iris.csv.dvc' didn't change, skipping                                  
Running stage 'train_model':                                                    
> python train.py
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


In [26]:
!git add dvc.lock
!git commit -m "Add lock file after running DVC pipeline"


[master c25d9a3] Add lock file after running DVC pipeline
 1 file changed, 18 insertions(+)
 create mode 100644 dvc.lock


In [27]:
!dvc push


Pushing
!
  0% Checking cache in 'week1_assignment_2_lab_demo_iitm_bs/files/md5'| |0/? [00
 50% Querying cache in 'week1_assignment_2_lab_demo_iitm_bs/files/md5'|▌|1/2 [00
                                                                                
!
  0% Checking cache in '/home/jupyter/.dvc/cache/files/md5'| |0/? [00:00<?,    ?
                                                                                
!
  0%|          |Pushing to gs                         0/1 [00:00<?,     ?file/s]

!

  0%|          |/home/jupyter/.dvc/cache/files/0.00/2.34k [00:00<?,        ?B/s]

                                                                                
100%|██████████|Pushing to gs                     1/1 [00:00<00:00,  8.79file/s]
Pushing                                                                         
1 file pushed
